In [1]:
import tensorflow as tf

In [3]:
def read_csv(batch_size, file_name, col_num):
    filename_queue = tf.train.string_input_producer([file_name])
    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(
        value,
        record_defaults=[[0] for i in range(col_num)])
    
    cols = tf.stack(decoded)
    print(cols)
    
    # batch actually reads the file and loads "batch_size" rows in a single tensor
    batch =  tf.train.shuffle_batch(cols,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size * 100) 
    id, labels, features = tf.split(batch, [1, 1, -1], 1)
    labels_onehot = tf.one_hot(labels, 2)
    return [features, labels_onehot]

features_train, labels_train = read_csv(100, "data/approve.csv", 541)

print(features_train)
print(labels_train)


# print(features)
# print(labels)

Tensor("stack_1:0", shape=(541,), dtype=int32)


TypeError: `Tensor` objects are not iterable when eager execution is not enabled. To iterate over this tensor use `tf.map_fn`.

In [ ]:
batch_size = 100
min_after_dequeue = 10000
capacity = min_after_dequeue + 3 * batch_size
    
example_batch, label_batch = tf.train.shuffle_batch(
        [features, labels], batch_size=batch_size, capacity=capacity,
        min_after_dequeue=min_after_dequeue)
print(example_batch)
print(label_batch)

In [ ]:

#define place holder
x = tf.placeholder(tf.float32, [None, 539])
y = tf.placeholder(tf.float32, [None, 2])

#create a 1-level neural network
W = tf.Variable(tf.truncated_normal([539, 2], stddev=0.1))
# b = tf.Variable(tf.zeros([2]))
b = tf.Variable(tf.zeros([2])+0.1)

wx_b = tf.matmul(x, W)+b

#激活函数，实现非线性化
# prediction = tf.nn.relu(wx_b)
# prediction = tf.nn.tanh(wx_b)
# prediction = tf.nn.sigmoid(wx_b)
prediction = tf.nn.softmax(wx_b)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
# loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=prediction))

#train
train = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

init = tf.global_variables_initializer()

#比较预测值与实际值
correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax 返回沿着某个维度最大值的位置
`
#准确率
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


with tf.Session() as sess:
    sess.run(init)
    
    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(1):
        # Retrieve a single instance:
        batch_xs, batch_ys = sess.run([example_batch, label_batch])
        print(batch_xs.shape)
        print(batch_xs)
    
        sess.run(train, feed_dict={x: batch_xs, y: batch_ys})


    coord.request_stop()
    coord.join(threads)

In [ ]:
x_data = np.linspace(-0.5, 0.5, 200)[:, np.newaxis]
noise = np.random.normal(0, 0.02, x_data.shape)
y_data = np.square(x_data) + noise

print(x_data)